Фаррахов Фанур 05-204 Вариант 18.

In [1]:
import math
import numpy as np
import sympy as sp
import scipy
import IPython.display as dp

**Вариант 18.**<br>
$f(x) = x^2 \sin(x)$<br>
$a = -1$, $b = 1$<br>

$x^{(1)} = -0.89, x^{(2)} = -0.42, x^{(3)} = -0.26$

In [2]:
x = sp.symbols('x')
f = x**2 * sp.sin(x)
a = -1
b = 1
X = [-0.89, -0.42, -0.26] # Контрольные точки x^(i)
Y = [f.subs(x, x_l) for x_l in X]

**Задача 1.**<br>

Для функции $f(x), x \in [a, b]$, построить интерполяционный сплайн первой степени по следующим узлам:
1. $x_k = a + k \frac{b-a}{4}$, $k = \overline{0, 4}$
2. $x_k = \frac{b -a}{2} \cos{\frac{(2k - 1)\pi}{10}} + \frac{b+a}{2}$, $k = \overline{1,5}$

In [3]:
# Первый случай
X1 = [(a + k * (b - a)/4) for k in range(5)]
Y1 = [f.subs(x, x_k) for x_k in X1]
print(f"x_k = {X1}")
print(f"y_k = {Y1}")

x_k = [-1.0, -0.5, 0.0, 0.5, 1.0]
y_k = [-0.841470984807897, -0.119856384651051, 0, 0.119856384651051, 0.841470984807897]


In [4]:
# Второой случай
X2 = [((b-a)/2 * np.cos((2*k-1)*np.pi / 10) + (b+a)/2) for k in range(1, 6)]   # x_k
Y2 = [f.subs(x, x_k) for x_k in X2]   # y_k
print(f"x_k = {X2}")
print(f"y_k = {Y2}")

x_k = [0.9510565162951535, 0.5877852522924731, 6.123233995736766e-17, -0.587785252292473, -0.9510565162951535]
y_k = [0.736296697723516, 0.191581718715425, 2.29584502165847e-49, -0.191581718715425, -0.736296697723516]


In [5]:
def spline_interpolate(X, Y, x):
    S = [((Y[k+1] - Y[k]) * (x - X[k+1]) / (X[k+1] - X[k]) + Y[k+1]) for k in range(len(X) - 1)]
    return np.array(S)[:, None]

def display(solution):
    string = "\\begin{cases}"
    for sol in solution:
        string += str(sp.latex(sol[0]))
        string += "\\\\"
    string += "\\end{cases}"
    dp.display(dp.Markdown(rf"${string}$"))

In [6]:
display(spline_interpolate(X1, Y1, x))

$\begin{cases}1.44322920031369 x + 0.601758215505795\\0.239712769302102 x\\0.239712769302102 x\\1.44322920031369 x - 0.601758215505795\\\end{cases}$

In [7]:
display(spline_interpolate(X2, Y2, x))

$\begin{cases}1.49947169783314 x - 0.689785631500853\\0.325938287781499 x - 1.99579640425591 \cdot 10^{-17}\\0.325938287781499 x - 5.55111512312578 \cdot 10^{-17}\\1.49947169783314 x + 0.689785631500853\\\end{cases}$

---
**Задача 2.1**<br>

Для функции $f(x), x \in [a, b]$ построить естественный сплайн третей степени, исползуя определение

In [8]:
def cubspline_interpolate(X, Y, x):
    M = __get_matrixes(X, Y)
    solution = np.linalg.solve(M[0], M[1])
    solution = np.reshape(solution, (len(X)-1, 4))

    return np.dot(solution, np.array([1, x, x**2, x**3]).T)[:, None]

def __get_matrixes(X, Y):
    nsegs = len(X) - 1 # Number of segments
    n = 4*(nsegs)
    cub   = lambda x, y: np.array([[1, x, x**2, x**3],
                                   [1, y, y**2, y**3]])
    dcub  = lambda x: np.array([0, 1, 2*x, 3*x**2, 0, -1, -2*x, -3*x**2])
    ddcub = lambda x: np.array([0, 0, 2, 6*x])

    # Добавление условий интерполяции
    int_rules = np.zeros((nsegs, 2, n))
    sample = np.zeros((2, n-4))
    for k in range(nsegs):
        int_rules[k] = np.insert(sample, 4 * k, cub(X[k], X[k+1]).T, axis=1)
    int_rules = np.reshape(int_rules, (int_rules.size))

    # Условия гладкой склейки на внутренних узловых точках
    sm_rules = np.zeros((nsegs-1, n))  # Smooth gluing rules
    sample = np.zeros((n-8, ))
    for k in range(nsegs-1):
        sm_rules[k] = np.insert(sample, 4*k, dcub(X[k+1]).T)

    # Условия дважды гладкой склейки на внутренних узловых точках
    dsm_rules = np.zeros((nsegs-1, n))
    for k in range(nsegs-1):
        dsm_rules[k] = np.insert(sample, 4*k, np.append(ddcub(X[k+1]), ddcub(-X[k+1])).T)
    
    # Добавление остаточных условий
    res = np.array(ddcub(X[0]))
    res.resize(2 * n - 4)
    res = np.append(res, ddcub(X[-1]))

    # Объединение всех условий в одну матрицу коэф СЛАУ
    int_rules = np.append(int_rules, [sm_rules, dsm_rules])
    res = np.append(int_rules, res)
    res.resize((n, n))

    # Добавление вектора-столбца свободных членов СЛАУ
    resY = np.zeros(n)
    for k in range(nsegs):
        resY[2*k] = Y[k]
        resY[2*k+1] = Y[k+1]
        
    return [res, resY]

In [9]:
solution1 = cubspline_interpolate(X1, Y1, x)
display(solution1)

$\begin{cases}0.802344287341059 x^{3} + 2.40703286202318 x^{2} + 3.6496759905016 x + 1.20351643101159\\0.80234428734106 x^{3} - 2.40703286202318 x^{2} - 1.16438973354475 x - 1.11022302462516 \cdot 10^{-16}\\0.802344287341061 x^{3} + 2.40703286202318 x^{2} - 1.16438973354475 x\\0.80234428734106 x^{3} - 2.40703286202318 x^{2} + 3.64967599050161 x - 1.20351643101159\\\end{cases}$

In [10]:
solution2 = cubspline_interpolate(X2, Y2, x)
display(solution2)

$\begin{cases}0.557735998005517 x^{3} - 1.59131536582658 x^{2} + 2.93930035099574 x - 1.09957187437619\\0.557735998005521 x^{3} + 1.59131536582659 x^{2} - 0.802106464121344 x + 5.55111512312578 \cdot 10^{-17}\\0.557735998005522 x^{3} - 1.59131536582659 x^{2} - 0.802106464121344 x + 1.66533453693773 \cdot 10^{-16}\\0.55773599800552 x^{3} + 1.59131536582659 x^{2} + 2.93930035099575 x + 1.09957187437619\\\end{cases}$

**Задача 2.2**

Для функции $f(x), x \in [a, b]$ построить естественный сплайн третей степени через метод моментов.

Вот система, которую будем решать:

$
\begin{cases}
\frac {(x_1 - x_0)^2}{2} + \frac{M_1 - M_0}{x_1 - x_0}  \frac{(x_1 - x_0)^3}{6} + G_0(x_1 - x_0) + f(x_0) = f(x_1) \\
\frac {(x_2 - x_1)^2}{2} + \frac{M_2 - M_1}{x_2 - x_1}  \frac{(x_2 - x_1)^3}{6} + G_1(x_2 - x_1) + f(x_1) = f(x_2) \\
\frac {(x_3 - x_2)^2}{2} + \frac{M_3 - M_2}{x_3 - x_2}  \frac{(x_3 - x_2)^3}{6} + G_2(x_3 - x_2) + f(x_2) = f(x_3) \\
\frac {(x_4 - x_3)^2}{2} + \frac{M_4 - M_3}{x_4 - x_3}  \frac{(x_4 - x_3)^3}{6} + G_3(x_4 - x_3) + f(x_3) = f(x_4) \\
M_0 (x_1 - x_0) + \frac{M_1 - M_0}{x_1 - x_0}  \frac{(x_1 - x_0)^2}{2} + G_0 = G_1 \\
M_1 (x_2 - x_1) + \frac{M_2 - M_1}{x_2 - x_1}  \frac{(x_2 - x_1)^2}{2} + G_1 = G_2 \\
M_2 (x_3 - x_2) + \frac{M_3 - M_2}{x_3 - x_2}  \frac{(x_3 - x_2)^2}{2} + G_2 = G_3 \\
M_0 = 0 \\
M_4 = 0
\end{cases}
$

In [20]:
def cubmom_interpolate(X, Y, x):
    M0, M1, M2, M3, M4 = sp.symbols('M0, M1, M2, M3, M4')
    G0, G1 ,G2, G3 = sp.symbols('G0, G1 ,G2, G3')
    M = [M0, M1, M2, M3, M4]
    G = [G0, G1 ,G2, G3]
    g   = lambda x, k : (x - X[k])**2 / 2 + (M[k+1] - M[k]) / (X[k+1] - X[k]) * (x - X[k])**3 / 6 + G[k] * (x - X[k]) + Y[k] - Y[k+1]
    dg  = lambda x, k : M[k] * (x - X[k]) + (M[k+1] - M[k]) / (X[k+1] - X[k]) * (x - X[k])**2 / 2 + G[k] - G[k+1]

    G_rules   = np.array([g(X[k+1], k) for k in range(4)])
    dG_rules  = np.array([dg(X[k+1], k) for k in range(3)])
    ddG_rules = np.array([M[0], M[4]])

    eqns = np.concatenate([G_rules, dG_rules, ddG_rules])
    sol = sp.solve(eqns, M + G)

    return [[sp.simplify(g(x, k).subs(sol)) + Y[k+1]] for k in range(4)]

In [58]:
solution3 = cubmom_interpolate(X1, Y1, x)
display(solution3)

$\begin{cases}- 2.75089469945506 x^{3} - 7.75268409836519 x^{2} - 5.37173122318773 x - 1.2114128090855\\0.687723674863766 x^{3} + 1.53158551229565 x^{2} + 0.833574606733984 x - 2.77555756156289 \cdot 10^{-17}\\15.1299208470029 x^{3} + \frac{x^{2}}{2} - 3.79276744244861 x\\- 13.0667498224116 x^{3} + 20.1001247336173 x^{2} - 5.84014571089209 x - 0.351758215505795\\\end{cases}$

In [59]:
solution4 = cubmom_interpolate(X2, Y2, x)
display(solution4)

$\begin{cases}- 2.92594443709311 x^{3} + 8.84821556964488 x^{2} - 6.82344878690547 x + 1.73950660212478\\- 0.836564663815663 x^{3} + 1.97516111593957 x^{2} - 0.546006304167796 x + 1.11022302462516 \cdot 10^{-16}\\12.5231279955347 x^{3} + 0.499999999999998 x^{2} - 3.70680339716316 x + 2.22044604925031 \cdot 10^{-16}\\- 15.9833122453065 x^{3} - 27.6842656617305 x^{2} - 12.1881491569533 x - 1.03672559789397\\\end{cases}$

**Задача 3**

Вычислить разницу между значениями функции и построенными сплайнами в контрольных точках $x^{(i)}, i = \overline{1,3}$

In [55]:
X = [-0.89, -0.42, -0.26]
Y = [f.subs(x, x_i) for x_i in X]

In [69]:
def diff_in_control_points(X, Y, sol):
    res = []
    for k in range(len(X)):
        if k != 2:
            res.append(np.abs(Y[k] - sol[0][0].subs(x, X[k])))
        else:
            res.append(np.abs(Y[k] - sol[1][0].subs(x, X[k])))

    return res

In [71]:
diff_in_control_points(X, Y, solution1)

[0.0881937892128213, 0.107737972220988, 0.143302551362472]